In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext,SparkConf
from datetime import datetime

In [ ]:
conf=SparkConf().setAppName("TestApp").setMaster("local[*]")
sc=SparkContext(conf=conf)

In [ ]:
productsRdd=sc.textFile("/content/products.csv")

In [ ]:
productsListRdd=productsRdd.map(lambda x:x.split(","))

In [ ]:
productsListRdd.take(5)

[['product_id', 'product_name', 'category'],
 ['P0001', 'Product_1', 'Toys'],
 ['P0002', 'Product_2', 'Electronics'],
 ['P0003', 'Product_3', 'Electronics'],
 ['P0004', 'Product_4', 'Toys']]

In [ ]:
headerProducts=productsListRdd.first()
productsListRdd1=productsListRdd.filter(lambda x:x !=headerProducts)

In [ ]:
productsListRdd1.take(5)

[['P0001', 'Product_1', 'Toys'],
 ['P0002', 'Product_2', 'Electronics'],
 ['P0003', 'Product_3', 'Electronics'],
 ['P0004', 'Product_4', 'Toys'],
 ['P0005', 'Product_5', 'Books']]

In [ ]:
pairProductRdd=productsListRdd1.map(lambda x:(x[0],x[2]))

In [ ]:
pairProductRdd.take(5)

[('P0001', 'Toys'),
 ('P0002', 'Electronics'),
 ('P0003', 'Electronics'),
 ('P0004', 'Toys'),
 ('P0005', 'Books')]

In [ ]:
salesRdd=sc.textFile("/content/sales.csv")

In [ ]:
salesListRdd=salesRdd.map(lambda x:x.split(","))

In [ ]:
headerSales=salesListRdd.first()
salesListRdd1=salesListRdd.filter(lambda x:x !=headerSales)

In [ ]:
salesListRdd1.take(5)

[['1', '103', 'P0363', '3', '98.89', '2025-01-20'],
 ['2', '436', 'P0088', '5', '179.75', '2024-12-17'],
 ['3', '349', 'P0139', '1', '96.37', '2024-09-27'],
 ['4', '271', 'P0120', '3', '42.19', '2025-01-22'],
 ['5', '107', 'P0313', '5', '30.67', '2025-02-19']]

In [ ]:
pairSalesRdd=salesListRdd1.map(lambda x:(x[2],x[3]))

In [ ]:
pairSalesRdd.take(5)

[('P0363', '3'),
 ('P0088', '5'),
 ('P0139', '1'),
 ('P0120', '3'),
 ('P0313', '5')]

In [ ]:
countQuantity=pairSalesRdd.map(lambda x:(x[0],int(x[1])))\
.reduceByKey(lambda x,y:x+y)

In [ ]:
countQuantity.take(5)

[('P0088', 3730),
 ('P0139', 3622),
 ('P0225', 3698),
 ('P0411', 3605),
 ('P0201', 3482)]

In [ ]:
joinRdd=countQuantity.join(pairProductRdd).map(lambda x:(x[1][1],x[1][0])).reduceByKey(lambda x,y:x+y)

In [ ]:
joinRdd.take(15)

[('Toys', 3730),
 ('Clothing', 3622),
 ('Toys', 3698),
 ('Toys', 3605),
 ('Electronics', 3482),
 ('Toys', 3591),
 ('Clothing', 3591),
 ('Books', 3540),
 ('Electronics', 3602),
 ('Clothing', 3751),
 ('Electronics', 3824),
 ('Electronics', 3553),
 ('Books', 3594),
 ('Toys', 3823),
 ('Toys', 3769)]

In [ ]:
joinRdd.saveAsTextFile('/content/most frequently')